In [1]:
# Results feature selection
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from preprocessing import complete_vars
from preprocessing import ratios
from preprocessing import breakdown_vars
from preprocessing import dummies_ohe
from preprocessing import Xy
from preprocessing import std_z
from imblearn.under_sampling import RandomUnderSampler
from Genfet import GaFeatureSelection
from Genfet import Geafet
from gentools import historical_fittest
from gentools import PerformanceTest
from gentools import get_names
from gentools import plot_stats
from ineq import Theil
from sklearn.linear_model import LogisticRegression

from Genfet import GaFeatureSelection
from Genfet import Geafet
from gentools import historical_fittest
from gentools import PerformanceTest
from gentools import get_names
from gentools import plot_stats
from ineq import Theil

df_train = pd.read_csv("Datapooled.csv")

VARS = ['Ganancia bruta', 'Ganancia (pérdida)','Ingresos de actividades ordinarias' , 'Costo de ventas', 'Patrimonio total',
     'Total pasivos', 'Total de activos', 'Ganancias acumuladas',  'Pasivos corrientes totales',  'Activos corrientes totales']

df_train.rename(columns={'Clasificación Industrial Internacional Uniforme Versión 4 A.C':'Sector'}, inplace=True)

df_train = df_train[VARS+[ 'event', 'Sector']]
print(df_train[df_train['event']==1].info())
df_train['complete-vars'] = complete_vars(df_train) #1 is that have all variables!
df_train =  df_train[df_train['complete-vars']==1] #filtering firms that have not financial information 
print(df_train[df_train['event']==1].info())
df_train = ratios(df_train)
predictors =[ 'GPM', 'NPM', 'ROE','ROA', 'IR', 'DER', 'RSL', 'CR', 'Ax1', 'Ax2', 'Sector']
print(df_train[df_train['event']==1].info())
df_train.replace([np.inf,-np.inf], np.nan, inplace=True)
df_train.dropna(inplace=True)
df_train.drop(columns=['complete-vars'], inplace=True)
df_train = df_train[predictors + ['event']].copy()
df_exp = df_train.copy()


X, y = Xy(df_train, 'event')
cat, nonormal, normal  = breakdown_vars(X)
nums = nonormal + normal
X = dummies_ohe(X, cat) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True, random_state = 666, stratify=y)
rus = RandomUnderSampler(random_state=123)
X_train, y_train = rus.fit_resample(X_train, y_train)
X_train = std_z(nums, X_train)
X_test = std_z(nums = nums, df_ = X_test)

<class 'pandas.core.frame.DataFrame'>
Index: 771 entries, 4 to 26499
Data columns (total 12 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Ganancia bruta                      353 non-null    float64
 1   Ganancia (pérdida)                  353 non-null    float64
 2   Ingresos de actividades ordinarias  353 non-null    float64
 3   Costo de ventas                     330 non-null    float64
 4   Patrimonio total                    353 non-null    float64
 5   Total pasivos                       353 non-null    float64
 6   Total de activos                    353 non-null    float64
 7   Ganancias acumuladas                351 non-null    float64
 8   Pasivos corrientes totales          352 non-null    float64
 9   Activos corrientes totales          353 non-null    float64
 10  event                               771 non-null    float64
 11  Sector                              771 non-null

c:\Users\sjrp2\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [2]:
classifier = LogisticRegression(penalty=None, max_iter=1500, random_state=666)

Pop_sizes  = [35, 50]  
generations = [25, 50 ]
Mutations = [0.01, 0.05]
Tournaments = [3, 7]
Cross = [0.9, 1]
probs_mate = [0.5, 0.7]
probs_mutate = [0.01 , 0.05]
regist= Geafet(
       pop_sizes = Pop_sizes,
       generations = generations,
       mutations = Mutations,
       tournaments_sizes = Tournaments,
       crossovers=Cross, 
       mate_indpb = probs_mate,
       mutate_indpb = probs_mutate,
       classifier = classifier,
       ineq_measure = Theil, 
       ineq_min  = 0,
       limit_unchanged= 50,
       X_train=X_train,
       y_train=y_train,
       k_folds = 2,
       verbose=True)


0.7125506072874493 0.0001864274255033926 [0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1] 0
0.7198011649537983 7.579572845761232e-05 [0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0] 0
0.7230795530483162 8.904694110598186e-05 [0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0] 0
0.7267043534363575 5.553159747908565e-05 [0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0] 1
0.7341384057064197 5.4028492169846956e-05 [1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1] 2
0.7387116948092558 3.828043893025761e-05 [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1] 0
0.7438585607940447 1.4434375419792083e-05 [1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1] 1
0.7438585607940447 8.483119684773851e-06 [1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 

c:\Users\sjrp2\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\sjrp2\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


0.7125506072874493 0.0001864274255033926 [0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1] 0
0.7198011649537983 7.579572845761232e-05 [0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0] 0
0.7230795530483162 8.995843213186727e-05 [0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0] 0
0.7267043534363575 5.495664642134908e-05 [0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0] 1
0.7291676836709788 4.828541041643491e-05 [0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0] 2
0.7291676836709788 3.2775482496485376e-05 [0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0] 3
0.7377576101342409 1.0458796499691483e-05 [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1] 4
0.7412955465587044 9.113526678636027e-06 [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 

c:\Users\sjrp2\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\sjrp2\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


0.7141873278236914 0.00015268633030916754 [0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1] 0
0.7262396694214877 0.00011654843275934683 [0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0] 0
0.7233606557377049 9.883965639858897e-05 [0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1] 0
0.7336337942097624 4.518901341184419e-05 [1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0] 1
0.7321866185786285 2.8533241526634202e-05 [0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0] 0
0.7335499337581226 1.506995286191183e-05 [0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0] 1
0.735520897918952 8.009990612100825e-06 [0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0] 2
0.7366948580248898 1.218292891462834e-05 [0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 

...

0.714360393603936 0.00012772538178404798 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0] 0
0.7294303797468354 7.986712377436805e-05 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1] 1
0.7337287104622872 2.6354579317154993e-05 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1] 0
0.739060731799321 1.1966300142232767e-05 [0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1] 1
0.739670275512313 4.300064827037282e-06 [0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1] 2
0.7426584853970746 1.6501346910618992e-05 [0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1] 3
0.7426584853970746 7.771934851116692e-06 [0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1] 4
0.7426584853970746 9.332807373583794e-06 [0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1

c:\Users\sjrp2\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\sjrp2\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


0.7144658788360035 9.575919598527504e-05 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0] 0
0.7225446839874701 9.146036810711362e-05 [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1] 1
0.7358550540368722 3.138865654253945e-05 [0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0] 0
0.7380585516178737 1.3455167465227706e-05 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0] 0
0.7386824802592254 6.980101931533393e-06 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1] 1
0.7386824802592254 2.080215254213371e-06 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1] 2
0.7386824802592254 1.2885467614755047e-06 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1] 3
0.7424136266679122 6.379772413607042e-08 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 

c:\Users\sjrp2\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\sjrp2\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


0.7144658788360035 9.78176088393311e-05 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0] 0
0.7225446839874701 9.146036810711362e-05 [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1] 1
0.7358550540368722 3.233983653469814e-05 [0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0] 0
0.7380585516178737 1.3886925563225596e-05 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0] 0
0.7386824802592254 6.752957888777322e-06 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1] 1
0.7386824802592254 1.6993721333257944e-05 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0] 2
0.7386824802592254 4.77996947017603e-06 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0] 3
0.7386824802592254 1.8937023757262876e-06 [0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1

In [3]:
[hyper for hyper in regist]

[(35, 25, 3, 0.01, 0.9, 0.5, 0.01),
 (35, 25, 3, 0.01, 0.9, 0.5, 0.05),
 (35, 25, 3, 0.01, 0.9, 0.7, 0.01),
 (35, 25, 3, 0.01, 0.9, 0.7, 0.05),
 (35, 25, 3, 0.01, 1, 0.5, 0.01),
 (35, 25, 3, 0.01, 1, 0.5, 0.05),
 (35, 25, 3, 0.01, 1, 0.7, 0.01),
 (35, 25, 3, 0.01, 1, 0.7, 0.05),
 (35, 25, 3, 0.05, 0.9, 0.5, 0.01),
 (35, 25, 3, 0.05, 0.9, 0.5, 0.05),
 (35, 25, 3, 0.05, 0.9, 0.7, 0.01),
 (35, 25, 3, 0.05, 0.9, 0.7, 0.05),
 (35, 25, 3, 0.05, 1, 0.5, 0.01),
 (35, 25, 3, 0.05, 1, 0.5, 0.05),
 (35, 25, 3, 0.05, 1, 0.7, 0.01),
 (35, 25, 3, 0.05, 1, 0.7, 0.05),
 (35, 25, 7, 0.01, 0.9, 0.5, 0.01),
 (35, 25, 7, 0.01, 0.9, 0.5, 0.05),
 (35, 25, 7, 0.01, 0.9, 0.7, 0.01),
 (35, 25, 7, 0.01, 0.9, 0.7, 0.05),
 (35, 25, 7, 0.01, 1, 0.5, 0.01),
 (35, 25, 7, 0.01, 1, 0.5, 0.05),
 (35, 25, 7, 0.01, 1, 0.7, 0.01),
 (35, 25, 7, 0.01, 1, 0.7, 0.05),
 (35, 25, 7, 0.05, 0.9, 0.5, 0.01),
 (35, 25, 7, 0.05, 0.9, 0.5, 0.05),
 (35, 25, 7, 0.05, 0.9, 0.7, 0.01),
 (35, 25, 7, 0.05, 0.9, 0.7, 0.05),
 (35, 25, 7, 0.0

In [4]:
posible = [regist[hyper][1] for hyper in regist][0]

In [5]:
[regist[hyper][0] for hyper in regist]

[0.7440644431882419,
 0.7440644431882419,
 0.7382528282401539,
 0.7426584853970746,
 0.747895860812763,
 0.747895860812763,
 0.7409059659307922,
 0.7409059659307922,
 0.7428951199443004,
 0.743563579277865,
 0.7484966505686244,
 0.7484966505686244,
 0.7423450809030746,
 0.7411586338972231,
 0.7376027558130431,
 0.7376027558130431,
 0.72754010270107,
 0.72754010270107,
 0.7393290813197596,
 0.7398263027295285,
 0.7423849243126351,
 0.7423849243126351,
 0.7311378562674795,
 0.7311378562674795,
 0.7276710005652911,
 0.7287868273295061,
 0.7393523155434731,
 0.7393523155434731,
 0.7423849243126351,
 0.7423849243126351,
 0.7311378562674795,
 0.7326036212226358,
 0.7440644431882419,
 0.7440644431882419,
 0.7382528282401539,
 0.7426584853970746,
 0.747895860812763,
 0.747895860812763,
 0.7409059659307922,
 0.7409059659307922,
 0.7428951199443004,
 0.743563579277865,
 0.7484966505686244,
 0.7484966505686244,
 0.7423450809030746,
 0.7484966505686244,
 0.7376027558130431,
 0.7376027558130431,
 0

In [6]:
np.array([regist[hyper][0] for hyper in regist]).argmax()

10

In [7]:
fittest_  = [regist[hyper][1] for hyper in regist][np.array([regist[hyper][0] for hyper in regist]).argmax()]

In [8]:
for sol in regist:
    print(PerformanceTest(solution = regist[sol][1],
                          classifier = classifier,
                          Xtrain=  X_train,
                          ytrain = y_train,
                          Xtest=  X_test,
                          ytest = y_test))

0.07342874922215308
0.07342874922215308
0.07204968944099378
0.06612466124661247
0.07257072570725707
0.07257072570725707
0.06541554959785523
0.06541554959785523
0.07142857142857142
0.06764866339334424
0.06783369803063458
0.06783369803063458
0.06612466124661247
0.0660530590146183
0.06751521859435529
0.06751521859435529
0.07103490508266994
0.07103490508266994
0.07173252279635259
0.07160194174757281
0.06946107784431138
0.06946107784431138
0.07645875251509054
0.07645875251509054
0.07292954264524104
0.07301980198019802
0.06528835690968444
0.06528835690968444
0.06946107784431138
0.06946107784431138
0.0774365821094793
0.07738492328218813
0.07342874922215308
0.07342874922215308
0.07204968944099378
0.06612466124661247
0.07257072570725707
0.07257072570725707
0.06541554959785523
0.06541554959785523
0.07142857142857142
0.06764866339334424
0.06783369803063458
0.06783369803063458
0.06612466124661247
0.06783369803063458
0.06751521859435529
0.06751521859435529
0.07103490508266994
0.07103490508266994
0.

In [9]:
PerformanceTest(solution = posible,
                          classifier = classifier,
                          Xtrain=  X_train,
                          ytrain = y_train,
                          Xtest=  X_test,
                          ytest = y_test)

0.07342874922215308

In [10]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import classification_report

# Forward Selection
sfs = SequentialFeatureSelector(estimator=classifier, n_features_to_select='auto', cv=2)
sfs.fit(X_train, y_train)
forwardvar = sfs.get_support()
sfs.get_feature_names_out()


# Backward Selection
sfs = SequentialFeatureSelector(estimator=classifier, n_features_to_select='auto', cv=2, direction='backward')
sfs.fit(X_train, y_train)
backwardvar = sfs.get_support()
sfs.get_feature_names_out()
X_train.columns[sfs.get_support()]


all = [1 for x in X_train.columns]
candidates = [all, fittest_, backwardvar, forwardvar]

# This function assest the performance of the model in testing 

for candidate in candidates:
    print(PerformanceTest(solution = candidate,
                         classifier = classifier,
                         Xtrain=  X_train,
                         ytrain=  y_train,
                         Xtest=  X_test,
                         ytest = y_test))

0.06723646723646724
0.06783369803063458
0.06548279689234185
0.0665938864628821


In [11]:
# Fittest 
vars = get_names(fittest_, X_train)
classifier.fit(X_train.loc[:,vars], y_train)
preds = classifier.predict( X_test.loc[:,vars ])
print(classification_report(y_test, preds))
selectGA  = pd.DataFrame(classification_report(y_test, preds , output_dict=True)).iloc[:,0:2]

              precision    recall  f1-score   support

         0.0       1.00      0.48      0.65      3296
         1.0       0.04      0.97      0.07        64

    accuracy                           0.49      3360
   macro avg       0.52      0.73      0.36      3360
weighted avg       0.98      0.49      0.64      3360



In [12]:

# Forward Selection
sfs = SequentialFeatureSelector(estimator=classifier, n_features_to_select='auto', cv=2)
sfs.fit(X_train, y_train)
forwardvar = sfs.get_support()
sfs.get_feature_names_out()
classifier.fit(X_train.loc[:,forwardvar], y_train)
preds_forward = classifier.predict(X_test.loc[:,forwardvar])
print(classification_report(y_test, preds_forward))
forward  = pd.DataFrame(classification_report(y_test, preds_forward , output_dict=True)).iloc[:,0:2]

              precision    recall  f1-score   support

         0.0       1.00      0.48      0.65      3296
         1.0       0.03      0.95      0.07        64

    accuracy                           0.49      3360
   macro avg       0.52      0.72      0.36      3360
weighted avg       0.98      0.49      0.64      3360



In [13]:
# Backward Selection
sfs = SequentialFeatureSelector(estimator=classifier, n_features_to_select='auto', cv=2, direction='backward')
sfs.fit(X_train, y_train)
backwardvar = sfs.get_support()
classifier.fit(X_train.loc[:,backwardvar], y_train)
preds_backward = classifier.predict(X_test.loc[:,backwardvar])
print(classification_report(y_test, preds_backward))
backward  = pd.DataFrame(classification_report(y_test, preds_backward , output_dict=True)).iloc[:,0:2]


              precision    recall  f1-score   support

         0.0       1.00      0.49      0.66      3296
         1.0       0.03      0.92      0.07        64

    accuracy                           0.50      3360
   macro avg       0.52      0.71      0.36      3360
weighted avg       0.98      0.50      0.65      3360



In [14]:
models_tab = pd.concat([selectGA, forward, backward], axis=1)
cols_names =  pd.MultiIndex.from_tuples([('Genetic selection','No-Default'),("Genetic selection",'Deafult'),
              ("Forward selection",'No-Default'),('Forward selection','Default'),
              ("Backward selection",'No-Default'),('Backward selection','Default')])
models_tab.columns  = cols_names
models_tab = models_tab.style.set_table_styles([
   {'selector': 'th','props': [('text-align', 'center')]}]).format(precision=2)
models_tab.to_latex("feature-selection-logistic.tex")
models_tab

In [15]:
vars

Index(['ROE', 'ROA', 'IR', 'RSL', 'Sector_D', 'Sector_E', 'Sector_G',
       'Sector_J', 'Sector_K', 'Sector_L', 'Sector_M', 'Sector_O', 'Sector_Q',
       'Sector_R'],
      dtype='object')

In [16]:
X_train.loc[:,forwardvar].columns

Index(['ROE', 'ROA', 'IR', 'RSL', 'Ax1', 'Sector_C', 'Sector_D', 'Sector_F',
       'Sector_H', 'Sector_I', 'Sector_J', 'Sector_O', 'Sector_R', 'Sector_U'],
      dtype='object')

In [17]:
X_train.loc[:,backwardvar].columns

Index(['GPM', 'NPM', 'ROE', 'IR', 'DER', 'RSL', 'CR', 'Ax1', 'Ax2', 'Sector_C',
       'Sector_F', 'Sector_G', 'Sector_J', 'Sector_P', 'Sector_U'],
      dtype='object')

In [18]:
y_test.value_counts()

event
0.0    3296
1.0      64
Name: count, dtype: int64

In [19]:
X_train.index

Index([11250, 14336,  9571,  3175,  8136,  3385,   822, 12247,  3985, 13209,
       ...
        4788,  4985,  2971,  6943,  9109,  1157,  3242,  9792,  9255,   987],
      dtype='int64', length=508)

In [20]:
y_test.index

Index([19146,  3820, 10870, 17643, 18621,  9242, 13245, 20033,   952,  4099,
       ...
        1161, 19710,  9821, 15427,  8818, 10170,  8837,  3652, 19586, 13580],
      dtype='int64', length=3360)